In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import string
import random
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.contrib import seq2seq

/home/florian/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
chars = string.ascii_letters + '>_'
char2idx = {ch: i for i, ch in enumerate(chars)}
idx2char = {i: ch for i, ch in enumerate(chars)}

# best weight values are saved in the checkpoint
with tf.Session() as sess:
    # restore the session that we used to train the model
    saver = tf.train.import_meta_graph('./saved_translator/pig_latin_encoder-decoder.ckpt.meta')
    saver.restore(sess, './saved_translator/pig_latin_encoder-decoder.ckpt')
    
    graph = tf.get_default_graph()
    
    #for op in graph.get_operations():
    #    if 'input' in str(op.name):
    #        print(str(op.name))

    
    #show_graph(graph)
    
    word_input = graph.get_tensor_by_name('encoder_input:0')
    predictions = graph.get_tensor_by_name('decoder_pred:0')
    
    val_input = [char2idx[c] for c in 'test']

    val_input = np.asarray(val_input).reshape(1, len(val_input))

    # for the inference mode we only pass the english word to translate
    prediction = sess.run(predictions, feed_dict={word_input: val_input})

    print("Given: {}, Predicted: {}".format('test', 
          ''.join([idx2char[idx] for idx in prediction[0]])))    
            
    relevant_nodes = ['encoder_input', 'decoder_pred']
    output_graph_def = tf.graph_util.convert_variables_to_constants(
        sess, # The session is used to retrieve the weights
        tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
        relevant_nodes # The output node names are used to select the usefull nodes
    )
    
    # Finally we serialize and dump the output graph to the filesystem
    with tf.gfile.GFile('./saved_translator/piglatin_enc-dec.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

INFO:tensorflow:Restoring parameters from ./saved_translator/pig_latin_encoder-decoder.ckpt
Given: test, Predicted: ettesay_
INFO:tensorflow:Froze 6 variables.
INFO:tensorflow:Converted 6 variables to const ops.


In [20]:
test_sentence = 'Convert this to piglatin please'

def load_graph(graph_flnm):
    with tf.gfile.GFile(graph_flnm, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    with tf.Graph().as_default() as graph:
        
        # the prefix will be import by default, so we'll give it something meaningful
        tf.import_graph_def(graph_def, name='enc-dec')

    return graph
    
graph = load_graph('./saved_translator/piglatin_enc-dec.pb')

with tf.Session(graph=graph) as sess:
    word_input = graph.get_tensor_by_name('enc-dec/encoder_input:0')
    predictions = graph.get_tensor_by_name('enc-dec/decoder_pred:0')
    
    output_translation = []
    for word in test_sentence.split():
    
        val_input = [char2idx[c] for c in word]
        val_input = np.asarray(val_input).reshape(1, len(val_input))

        # for the inference mode we only pass the english word to translate
        prediction = sess.run(predictions, feed_dict={word_input: val_input})

         
        output_translation.append(''.join([idx2char[idx] for idx in prediction[0]]))
        
print(' '.join(output_translation).replace('_', ''))

onetarytay hitesay teonay ipgliatinray lepasesay
